In [1]:
import wandb
wandb.login()

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: taoprajjwal (pj-runs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
import torch
from tqdm import tqdm
import math
device = torch.device("cuda")

def generate_low_inner_product_vectors(n, epsilon, device='cpu', seed=None):
    if seed is not None:
        torch.manual_seed(seed)
        
    # Compute k = exp(epsilon^2 * n / 4)
    k = int(math.exp((epsilon**2 * n) / 4))
    
    # Generate k x n random ±1 vectors
    signs = torch.randint(0, 2, (k, n), device=device, dtype=torch.float32) * 2 - 1  # ±1
    vectors = signs / math.sqrt(n)  # Normalize each to have norm 1

    # Compute inner products
    ip_matrix = torch.matmul(vectors, vectors.T)
    mask = torch.eye(k, device=device).bool()
    ip_matrix[mask] = 0  # zero out diagonal

    max_ip = ip_matrix.abs().max().item()
    success = (ip_matrix.abs() < epsilon).all().item()

    print(f"Generated {k} vectors in {n} dimensions.")
    print(f"Maximum off-diagonal inner product: {max_ip:.4f}")
    print("All inner products < ε:", success)

    return vectors


In [3]:
vecs = generate_low_inner_product_vectors(1000, 0.2)

Generated 22026 vectors in 1000 dimensions.
Maximum off-diagonal inner product: 0.1880
All inner products < ε: True


In [4]:
torch.linalg.norm(vecs, dim=1)

tensor([1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000])

In [5]:
proj = torch.randn(1000, 500)

In [6]:
stud_vec_2 = torch.randn(22026, 500)
stud_vec_2 = stud_vec_2 / stud_vec_2.norm(dim=1, keepdim=True)
stud_vec_2_init = stud_vec_2.data

In [7]:
stud_vec_2

tensor([[ 0.0294, -0.0400,  0.0588,  ...,  0.0399, -0.0438, -0.0007],
        [ 0.0367,  0.0058,  0.0245,  ...,  0.0360,  0.0393, -0.0008],
        [-0.0168,  0.0031, -0.0716,  ...,  0.1032, -0.0077, -0.0650],
        ...,
        [ 0.0692, -0.0596, -0.0078,  ...,  0.0598, -0.0692, -0.0877],
        [ 0.0233, -0.0206, -0.0192,  ...,  0.0172,  0.0116,  0.0221],
        [-0.0042, -0.0105,  0.1012,  ..., -0.0878,  0.0217, -0.0182]])

In [8]:
stud_vec_2 = stud_vec_2.requires_grad_(True)

In [9]:
stud_vecs = vecs @ proj
stud_vecs = stud_vecs / stud_vecs.norm(dim=1, keepdim=True)

In [10]:
student_vecs = stud_vecs.requires_grad_(True)

In [11]:
def luby_mis_from_dense(adj_dense, max_iters=1000, device=device):
    """
    Compute an approximate maximal independent set using Luby's algorithm.
    
    Args:
        adj_dense (torch.Tensor): Dense n x n adjacency matrix with 1s as edges, 0s elsewhere.
        max_iters (int): Maximum iterations to try.
        device (str): 'cpu' or 'cuda'.
    
    Returns:
        torch.Tensor: Indices of nodes in the MIS.
    """
    # Convert to sparse format for efficiency
    adj_dense = adj_dense.to(device)
    n = adj_dense.size(0)
    adj_sparse = adj_dense.to_sparse().coalesce()

    in_set = torch.zeros(n, dtype=torch.bool, device=device)
    remaining = torch.ones(n, dtype=torch.bool, device=device)

    for _ in range(max_iters):
        if not remaining.any():
            break

        priorities = torch.rand(n, device=device)
        priorities[~remaining] = -1e9  # Effectively ignore these

        # Compute max neighbor priority
        neighbor_max = torch.sparse.mm(adj_sparse, priorities.unsqueeze(1)).squeeze(1)

        selected = (priorities > neighbor_max) & remaining
        in_set[selected] = True

        # Mark neighbors of selected for removal
        selected_mask = selected.to(adj_dense.dtype).unsqueeze(0)  # (1 x n)
        neighbors_of_selected = (selected_mask @ adj_dense).squeeze(0).bool()

        to_remove = selected | neighbors_of_selected
        remaining[to_remove] = False

    return torch.nonzero(in_set, as_tuple=False).squeeze(1)

In [12]:
def get_adjc(vec, eps):
    norm_vec = vec / vec.norm(dim=1, keepdim=True)
    ip_matrix = vec@vec.T
    mask = torch.eye(len(vec)).bool().to(device)
    ip_matrix[mask] = 0

    return (ip_matrix.abs() < eps).float()

In [ ]:
from importlib import reload
reload(sim)

In [13]:
import similarity_measures as sim
import similarity

In [14]:
lin = sim.LinearMeasure(approx=True)
cka = sim.CKA(biased=False)
measure = similarity.make("measure/netrep/procrustes-distance=euclidean")

In [15]:
from torch.utils.data import TensorDataset, DataLoader
def copy_and_clone(stud_vec):
    stud_vec_2 = stud_vec.clone().detach()
    stud_vec_2.requires_grad_(True)
    dataset = TensorDataset(vecs, stud_vec_2)
    return stud_vec_2, dataset
    

In [15]:
def get_kernel_frobenius(x_vec, y_vec):
    k_x = x_vec@ x_vec.T
    k_y = y_vec@ y_vec.T

    return torch.norm(k_x -k_y, p="fro")

In [16]:
len(luby_mis_from_dense(1- get_adjc(stud_vec_2, 0.2)))

20597

In [17]:
len(luby_mis_from_dense(1- get_adjc(stud_vecs, 0.2)))

10381

In [ ]:
stud_vec_2_cka, dataset_cka = copy_and_clone(stud_vecs)
dl = DataLoader(dataset_cka, batch_size=256, shuffle=False)
optim = torch.optim.AdamW([stud_vec_2_cka])
wandb.init(project="synthetic_minimization", entity="pj-runs", name="cka_minim_projected_stud")
for epoch in range(10):
    for batch in tqdm(dl, desc=f"Epoch {epoch+1}"):
        a,b = batch
        loss = 1- (cka(b.unsqueeze(1), a.unsqueeze(1)))
        shape = lin(b.unsqueeze(1), a.unsqueeze(1))
        k_frob = get_kernel_frobenius(a,b)
        adj = get_adjc(stud_vec_2_cka, 0.2)
        s = luby_mis_from_dense(1-adj)
        wandb.log( {"shape": shape.item(), "cka":loss.item(), "eps-orth-vectors": len(s), "kernel_frob":k_frob})
        loss.backward()
        optim.step()
        optim.zero_grad()
wandb.finish()

In [ ]:
stud_vec_2_shape, dataset_shape = copy_and_clone(stud_vecs)
dl = DataLoader(dataset_shape, batch_size=256, shuffle=False)
optim = torch.optim.AdamW([stud_vec_2_shape])
wandb.init(project="synthetic_minimization", entity="pj-runs", name="shape_minim_projected_stud")
for epoch in range(10):
    for batch in tqdm(dl, desc=f"Epoch {epoch+1}"):
        a,b = batch
        ck = 1- (cka(b.unsqueeze(1), a.unsqueeze(1)))
        loss = lin(b.unsqueeze(1),  a.unsqueeze(1))
        adj = get_adjc(stud_vec_2_shape, 0.2)
        k_frob = get_kernel_frobenius(a,b)
        s = luby_mis_from_dense(1-adj)
        wandb.log( {"shape": loss.item(), "cka":ck.item(), "eps-orth-vectors": len(s) ,"kernel_frob":k_frob})
        loss.backward()
        optim.step()
        optim.zero_grad()
wandb.finish()

In [ ]:
stud_vec_2_kfrob, dataset_kfrob = copy_and_clone(stud_vecs)
dl = DataLoader(dataset_kfrob, batch_size=256, shuffle=False)
optim = torch.optim.AdamW([stud_vec_2_kfrob])
wandb.init(project="synthetic_minimization", entity="pj-runs", name="kernel_frobenius_diff_projected_stud")
for epoch in range(10):
    for batch in tqdm(dl, desc=f"Epoch {epoch+1}"):
        a,b = batch
        ck = 1- (cka(b.unsqueeze(1), a.unsqueeze(1)))
        shape = lin(b.unsqueeze(1),  a.unsqueeze(1))
        adj = get_adjc(stud_vec_2_kfrob, 0.2)
        loss = get_kernel_frobenius(a,b)
        s = luby_mis_from_dense(1-adj)
        print({"shape": shape.item(), "cka":ck.item(), "eps-orth-vectors": len(s), "kernel_frob": loss.item()})
        wandb.log( {"shape": shape.item(), "cka":ck.item(), "eps-orth-vectors": len(s), "kernel_frob": loss.item()})
        loss.backward()
        optim.step()
        optim.zero_grad()
wandb.finish()

In [ ]:
wandb.finish()

In [27]:
stud_vec_2_kfrob, dataset_kfrob = copy_and_clone(stud_vecs)

In [28]:
len(luby_mis_from_dense(1- get_adjc(stud_vec_2_kfrob, 0.2)))

10572

In [ ]:
stud_vec_2_cka, dataset_cka = copy_and_clone(stud_vecs)
dl = DataLoader(dataset_cka, batch_size=256, shuffle=False)
optim = torch.optim.AdamW([stud_vec_2_cka])
wandb.init(project="synthethic_minimization_batch", entity="pj-runs", name="cka_minim_projected_stud")
for epoch in range(100):
    a,b = dataset_cka[:]
    loss = 1- (cka(b.unsqueeze(1), a.unsqueeze(1)))
    shape = lin(b.unsqueeze(1), a.unsqueeze(1))
    k_frob = get_kernel_frobenius(a,b)
    adj = get_adjc(stud_vec_2_cka, 0.2)
    s = luby_mis_from_dense(1-adj)
    wandb.log( {"shape": shape.item(), "cka":loss.item(), "eps-orth-vectors": len(s), "kernel_frob":k_frob})
    loss.backward()
    optim.step()
    optim.zero_grad()
wandb.finish()

In [ ]:
stud_vec_2_shap, dataset_shap = copy_and_clone(stud_vecs)
dl = DataLoader(dataset_shap, batch_size=256, shuffle=False)
optim = torch.optim.AdamW([stud_vec_2_shap])
wandb.init(project="synthethic_minimization_batch", entity="pj-runs", name="shape_minim_projected_stud")
for epoch in range(100):
    a,b = dataset_shap[:]
    ck = 1- (cka(b.unsqueeze(1), a.unsqueeze(1)))
    loss = lin(b.unsqueeze(1), a.unsqueeze(1))
    k_frob = get_kernel_frobenius(a,b)
    adj = get_adjc(stud_vec_2_shap, 0.2)
    s = luby_mis_from_dense(1-adj)
    wandb.log( {"shape": loss.item(), "cka":ck.item(), "eps-orth-vectors": len(s), "kernel_frob":k_frob})
    loss.backward()
    optim.step()
    optim.zero_grad()
wandb.finish()

In [ ]:
stud_vec_2_kfrob, dataset_kfrob = copy_and_clone(stud_vecs)
dl = DataLoader(dataset_kfrob, batch_size=256, shuffle=False)
optim = torch.optim.AdamW([stud_vec_2_kfrob])
wandb.init(project="synthethic_minimization_batch", entity="pj-runs", name="kfrob_minim_projected_stud")
for epoch in range(100):
    a,b = dataset_kfrob[:]
    ck = 1- (cka(b.unsqueeze(1), a.unsqueeze(1)))
    shape = lin(b.unsqueeze(1), a.unsqueeze(1))
    loss = get_kernel_frobenius(a,b)
    adj = get_adjc(stud_vec_2_kfrob, 0.2)
    s = luby_mis_from_dense(1-adj)
    wandb.log( {"shape": shape, "cka":ck.item(), "eps-orth-vectors": len(s), "kernel_frob":loss.item()})
    loss.backward()
    optim.step()
    optim.zero_grad()
wandb.finish()

In [35]:
22026*22026 - success.sum()

tensor(7328.)

In [36]:
7328/2

3664.0

In [ ]:
optim = torch.optim.AdamW([stud_vec_2])
wandb.init(project="synthetic_minimization", entity="pj-runs", name="cka_minim")
for epoch in range(20):
    for batch in dl:
        a,b = batch
        loss = 1- (cka(b.unsqueeze(1), a.unsqueeze(1)))
        shape = measure(b.detach().numpy(), a.detach().numpy())
        wandb.log( {"shape": shape, "cka":loss.item()})
        loss.backward()
        optim.step()
        optim.zero_grad()

In [164]:
loss

tensor(0.2200, grad_fn=<RsubBackward1>)

In [106]:
lin(a.unsqueeze(1), b.unsqueeze(1))

tensor(0.2626, grad_fn=<SubBackward0>)

In [107]:
measure(b.detach().numpy(), a.detach().numpy()) **2

0.262740774754418

In [75]:
mx, wx = lin.partial_fit(b.unsqueeze(1), center_col_idx=0)

In [85]:
wx.shape

torch.Size([20, 1, 500])

In [ ]:
wx.mean(dim=0)[0]

## Linear combo

In [19]:
import random

In [20]:
vecs.shape

torch.Size([22026, 1000])

In [21]:
vecs[torch.randint(0, len(vecs), (10,))].sum(dim=0, keepdim=True).shape

torch.Size([1, 1000])

In [ ]:
SAMPLE_SIZE = 10000
k_max = 10
idx_list= []
group_ids = []
for i in range(SAMPLE_SIZE):
    k = random.randint(1,k_max)
    ix = torch.randint(0, len(vecs), (k,))
    idx_list.append(ix)
    group_ids.append(torch.full_like(ix, i))

In [54]:
from torch.utils.data import TensorDataset, DataLoader
def copy_and_clone(stud_vec):
    stud_vec_2 = stud_vec.clone().detach()
    stud_vec_2.requires_grad_(True).to(device)
    group_ids_tensor = torch.cat(group_ids)
    idx_tensor = torch.cat(idx_list)
    X_t = torch.zeros(len(idx_list), vecs.shape[1])
    
    X_t = X_t.index_add(0, group_ids_tensor, vecs[idx_tensor]).to(device)
    
    Y_t = torch.zeros(len(idx_list), stud_vec_2.shape[1])
    
    Y_t = Y_t.index_add(0, group_ids_tensor, stud_vec_2[idx_tensor]).to(device)
    
    dataset = TensorDataset(X_t, Y_t)
    return stud_vec_2, dataset
    

In [66]:
stud_vec_2_cka, dataset_cka = copy_and_clone(stud_vecs)
dl = DataLoader(dataset_cka, batch_size=4, shuffle=False)


In [67]:
a, b = dataset_cka[:]

In [73]:
stud_vec_2_kfrobime

In [ ]:
for batch in (dl):
    a,b = batch
    loss= 1- (cka(b.unsqueeze(1), a.unsqueeze(1)))
    print(loss)
    st = time.time()
    loss.backward()
    end = time.time()
    print(st-end)

In [ ]:
stud_vec_2_cka, dataset_cka = copy_and_clone(stud_vecs)
dl = DataLoader(dataset_cka, batch_size=256, shuffle=False)
optim = torch.optim.AdamW([stud_vec_2_cka])
wandb.init(project="linear_combo_2", entity="pj-runs", name="cka")
for epoch in range(10):
    for batch in tqdm(dl, desc=f"Epoch {epoch+1}"):
        a,b = batch
        loss = 1- (cka(b.unsqueeze(1), a.unsqueeze(1)))
        shape = lin(b.unsqueeze(1), a.unsqueeze(1))
        k_frob = get_kernel_frobenius(a,b)
        adj = get_adjc(stud_vec_2_cka, 0.2)
        s = luby_mis_from_dense(1-adj)
        wandb.log( {"shape": shape.item(), "cka":loss.item(), "eps-orth-vectors": len(s), "kernel_frob":k_frob})
        print( {"shape": shape.item(), "cka":loss.item(), "eps-orth-vectors": len(s), "kernel_frob":k_frob})
        loss.backward(retain_graph=True)
        optim.step()
        optim.zero_grad()
wandb.finish()

In [ ]:
wandb.finish()

In [ ]:
stud_vec_2_shape, dataset_shape = copy_and_clone(stud_vecs)
dl = DataLoader(dataset_shape, batch_size=256, shuffle=False)
optim = torch.optim.AdamW([stud_vec_2_shape])
wandb.init(project="linear_combo_2", entity="pj-runs", name="shape")
for epoch in range(10):
    for batch in tqdm(dl, desc=f"Epoch {epoch+1}"):
        a,b = batch
        ck = 1- (cka(b.unsqueeze(1), a.unsqueeze(1)))
        loss = lin(b.unsqueeze(1),  a.unsqueeze(1))
        adj = get_adjc(stud_vec_2_shape, 0.2)
        k_frob = get_kernel_frobenius(a,b)
        s = luby_mis_from_dense(1-adj)
        wandb.log( {"shape": loss.item(), "cka":ck.item(), "eps-orth-vectors": len(s) ,"kernel_frob":k_frob})
        loss.backward(retain_graph=True)
        optim.step()
        optim.zero_grad()
wandb.finish()

In [ ]:
wandb.finish()

In [58]:
stud_vec_2_kfrob, dataset_kfrob = copy_and_clone(stud_vecs)
dl = DataLoader(dataset_kfrob, batch_size=256, shuffle=False)
optim = torch.optim.AdamW([stud_vec_2_kfrob])
wandb.init(project="linear_combo_2", entity="pj-runs", name="kfrob")
for epoch in range(10):
    for batch in tqdm(dl, desc=f"Epoch {epoch+1}"):
        a,b = batch
        ck = 1- (cka(b.unsqueeze(1), a.unsqueeze(1)))
        shape = lin(b.unsqueeze(1),  a.unsqueeze(1))
        adj = get_adjc(stud_vec_2_kfrob, 0.2)
        loss = get_kernel_frobenius(a,b)
        s = luby_mis_from_dense(1-adj)
        print({"shape": shape.item(), "cka":ck.item(), "eps-orth-vectors": len(s), "kernel_frob": loss.item()})
        wandb.log( {"shape": shape.item(), "cka":ck.item(), "eps-orth-vectors": len(s), "kernel_frob": loss.item()})
        loss.backward(retain_graph=True)
        optim.step()
        optim.zero_grad()
wandb.finish()

Epoch 6:  43%|████▎     | 34/79 [00:44<00:58,  1.30s/it]

{'shape': 146.529541015625, 'cka': 0.40882694721221924, 'eps-orth-vectors': 5327, 'kernel_frob': 62.265689849853516}


Epoch 6:  44%|████▍     | 35/79 [00:45<00:57,  1.30s/it]

{'shape': 149.734130859375, 'cka': 0.41761285066604614, 'eps-orth-vectors': 1380, 'kernel_frob': 65.24884796142578}


Epoch 6:  46%|████▌     | 36/79 [00:47<00:58,  1.36s/it]

...

{'shape': 144.8671875, 'cka': 0.4162530303001404, 'eps-orth-vectors': 0, 'kernel_frob': 62.95224380493164}


Epoch 6:  61%|██████    | 48/79 [01:19<01:26,  2.79s/it]

{'shape': 144.01904296875, 'cka': 0.4200577735900879, 'eps-orth-vectors': 0, 'kernel_frob': 63.00102996826172}


Epoch 6:  61%|██████    | 48/79 [01:21<00:52,  1.69s/it]


KeyboardInterrupt: 

In [64]:
stud_vec_2_kfrob.norm(dim=1)

tensor([1.2220, 1.2900, 1.1977,  ..., 1.2642, 1.2866, 1.2407],
       grad_fn=<LinalgVectorNormBackward0>)

In [ ]:
wandb.finish()

In [63]:
X_t.requires_grad

False

In [58]:
stud_vecs.requires_grad

True